1. find flask ids in user_items
2. cross-reference these ids with item_data
3. determine amount of flasks available
4. cross-reference choice id from flasks in item_data with item id in item_data
5. determine amount of potential shards for each champion
6. get current shards for each champion owned
7. get level and necessary amount to upgrade each champion owned
8. determine which champions can be leveled up and by using which flasks
9. make exceptions for lev (very optional)
10. turn this into a standalone .py and make it automated with pulling the data when loading kong

In [154]:
# imports
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET

In [155]:
# parse data into data structures

user_items_raw = pd.read_json("user_items.json", orient="index")
# print(user_items_raw.head())

item_data_raw = pd.read_json("item_data.json", orient="index")
# print(item_data_raw.head())

user_units_raw = pd.read_json("user_units.json", orient="index")
# print(user_units_raw.head())

# use ET for cards_shard.xml ?

In [156]:
# constants for id boundaries, easier to change here than in-line
flasks_id_min = 2534
flasks_id_max = 2699

# reward, aether, chaos, wyld | f2p, p2w?
champion_id_min = [4000, 5600, 6600, 7600]
champion_id_max = [4999, 5999, 6999, 7999]

# shard ids are auto-generated in-game as well
champion_shards_id_min = []
champion_shards_id_max = []
for id_min, id_max in zip(champion_id_min, champion_id_max):
    champion_shards_id_min.append(id_min + 100000)
    champion_shards_id_max.append(id_max + 100000)

# constant champ level requirements
champion_costs = [50, 75, 100, 150, 225, 300, 400, 600, 900, 1200]
champion_costs_mythic = [150, 200, 250, 300, 450, 600, 750, 900, 1100, 1300]

In [157]:
# 1 pd dataframe with id, name, icon, desc, category, bundle etc. of available flasks in-game
item_data_flasks = item_data_raw[item_data_raw["id"].between(flasks_id_min,flasks_id_max)]

# keep only the specified columns and drop all others
item_data_flasks = item_data_flasks.drop(item_data_flasks.columns.difference(["id", "name", "desc"]), axis=1)

In [158]:
# ? pd dataframe with id and number of 
user_items_flasks = user_items_raw[user_items_raw["id"].between(flasks_id_min, flasks_id_max)]

In [159]:
# 6 pd dataframe with id and number of champion shards in inventory
user_items_champion_shards = []

# append multiple dataframes to lists
for id_min, id_max in zip(champion_shards_id_min, champion_shards_id_max):
    user_items_champion_shards.append(user_items_raw[user_items_raw["id"].between(id_min, id_max)])

# # concat dataframes into one
user_items_champion_shards = pd.concat(user_items_champion_shards)

In [160]:
# 7 pd dataframe with unit_id and level of all owned champions
user_units_champions_owned = []

# append multiple dataframes to lists 
for id_min, id_max in zip(champion_id_min, champion_id_max):
    user_units_champions_owned.append(user_units_raw[user_units_raw["unit_id"].between(id_min, id_max)])
    
# concat dataframes into one
user_units_champions_owned = pd.concat(user_units_champions_owned)
    
# keep only the specified columns and drop all others
user_units_champions_owned = user_units_champions_owned.drop(user_units_champions_owned.columns.difference(["unit_id", "level"]), axis=1)